In [2]:
import numpy as np
import torch
import torch.nn as nn
from process_data import sample
from basic_model import DeepModel_multi, gradients
from visual_data import matplotlib_vision
from matplotlib import ticker
import time
from tqdm import trange
import matplotlib.pyplot as plt


F:\Programs\Anaconda\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:

class Net(DeepModel_multi):
    def __init__(self, planes):
        super(Net, self).__init__(planes, active=nn.GELU())
        self.alpha_max, self.alpha_min = 2.5 * 10 ** 4, 0  # 2.5 / 10 ** 4
        self.q = 0.1
        self.GAMMA = 0.9

    ########################强制边界的输出转换，L_b, ########################
    def output_transform(self, inputs, outputs):
        x, y = inputs[..., (0,)], inputs[..., (1,)]
        bc = 16 * x * (1 - x) * y * (1 - y)# 在x=0, x=1, y=1, y=0上保证u=u0, v=0
        # u
        u0 = 1
        u = torch.abs(u0 + bc * outputs[..., (0,)])
        # v
        v = bc * outputs[..., (1,)]
        # p
        p = (1 - x) * outputs[..., (2,)]# 在x=1上p=0
        # rho
        # rho = tf.math.exp(-bc * tf.math.square(outputs[:, 3:]))
        # rho = 1 + bc * outputs[:, 3:]
        center = torch.square(x - 0.5) + torch.square(y - 0.5)
        # rho = center * outputs[:, 3:]
        rho = center * (
                bc * outputs[..., (3,)] + (1 - bc) * (1 + 1e-6 / 0.25) / (center + 1e-6)
        )
        rho = torch.maximum(torch.minimum(torch.ones_like(rho), rho), torch.zeros_like(rho))
        return torch.cat((u, v, p, rho), dim=-1)

    ########################方程参数： 质量力的alpha系数######################
    def alpha(self, rho):
        return self.alpha_max + (self.alpha_min - self.alpha_max) * rho * (1 + self.q) / (rho + self.q)

    ########################动量和连续方程残差 F ############################
    def equation(self, inn_var, out_var):
        u, v, p, rho = out_var[:, (0,)], out_var[:, (1,)], out_var[:, (2,)], out_var[:, (3,)]
        duda = gradients(u, inn_var)
        dudx, dudy = duda[:, (0,)], duda[:, (1,)]
        d2udx2 = gradients(dudx, inn_var)[:, (0,)]
        d2udy2 = gradients(dudy, inn_var)[:, (1,)]

        dvda = gradients(v, inn_var)
        dvdx, dvdy = dvda[:, (0,)], dvda[:, (1,)]
        d2vdx2 = gradients(dvdx, inn_var)[:, (0,)]
        d2vdy2 = gradients(dvdy, inn_var)[:, (1,)]

        dpda = gradients(p, inn_var)
        dpdx, dpdy= dpda[:, (0,)], dpda[:, (1,)]

        fx, fy = self.alpha(rho) * u, self.alpha(rho) * v
        eq1 = (-(d2udx2 + d2udy2) + dpdx - fx) * 0.01
        eq2 = (-(d2vdx2 + d2vdy2) + dpdy - fy) * 0.01
        eq3 = (dudx + dvdy) * 100.

        return torch.cat([eq1, eq2, eq3], dim=-1), duda, dvda

    ######################## 不等式残差，h: 密度积分应该大于0.9 ###############
    ## 该函数直接输出全域的一个积分值
    def inequal_constrain(self, y):

        return torch.square(torch.maximum(torch.tensor(0.0, dtype=torch.float32).to(device), torch.mean(y[:, -1]) - self.GAMMA))

    ######################## 优化目标，J: 耗散功 ############################
    ## if_reduced 输出全域的平均值 ，否则 各点耗散功
    def optim_func(self, inn_var, out_var, duda, dvda, if_reduced=False):
        p1 = torch.sum(torch.square(duda) + torch.square(dvda), dim=1)# keepdim, 保留缩减的维度为1
        p2 = self.alpha(out_var[:, 3]) * torch.sum(torch.square(out_var[:, :2]), dim=1)
        if if_reduced:
            return torch.mean(0.5 * (p1 + p2))
        else:
            return 0.5 * (p1 + p2)


In [4]:
import os
res_path = 'hpinn_stokes\\'
isCreated = os.path.exists(res_path)
if not isCreated:
    os.makedirs(res_path)

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

def gen_dataset(bounds, N_sample, method="pseudo", ndim=1):

    if ndim == 0:
        return bounds[0] * np.ones([N_sample, 1])
    elif ndim == 1:
        if method == 'uniform':
            return np.linspace(bounds[0], bounds[1], N_sample)
        else:
            X = sample(N_sample, ndim, method)
            return X * (np.array(bounds)[1]-np.array(bounds)[0]) + np.array(bounds)[0]
    elif ndim == 2:
        if method == 'uniform':
            n = int(np.sqrt(N_sample))
            x0 = np.linspace(bounds[0, 0], bounds[0, 1], n)
            x1 = np.linspace(bounds[1, 0], bounds[1, 1], n)
            X = np.concatenate([np.tile(x0, 1, n), np.tile(x1, 1, n)], axis=-1)
            return np.reshape(X, [-1, 2])
        else:
            X = sample(N_sample, ndim, method)
            return X * (np.array(bounds)[:, 1]-np.array(bounds)[:, 0]) + np.array(bounds)[:, 0]

# x [-1, 1] t [0, 1]
N_inner, N_bound = 64*64, 64
bound_x, bound_y = [0, 1], [0, 1]
x_inner = gen_dataset([bound_x, bound_y], N_inner, method='Sobol', ndim=2)
X_bond11 = np.concatenate([np.ones([N_bound, 1]) * bound_x[0], gen_dataset(bound_y, N_bound, method='Sobol', ndim=1)], axis=1).astype(dtype=np.float32)
X_bond12 = np.concatenate([np.ones([N_bound, 1]) * bound_x[1], gen_dataset(bound_y, N_bound, method='Sobol', ndim=1)], axis=1).astype(dtype=np.float32)
X_bond21 = np.concatenate([gen_dataset(bound_x, N_bound, method='Sobol', ndim=1), np.ones([N_bound, 1]) * bound_y[0]], axis=1).astype(dtype=np.float32)
X_bond22 = np.concatenate([gen_dataset(bound_x, N_bound, method='Sobol', ndim=1), np.ones([N_bound, 1]) * bound_y[1]], axis=1).astype(dtype=np.float32)

input = np.concatenate([X_bond11, X_bond12, X_bond21, X_bond22, x_inner,], axis=0).astype(np.float32)
field = np.zeros_like(input[:, (0, )]) # 未用到

input_train = torch.tensor(input, dtype=torch.float32).to(device)
field_train = torch.tensor(field, dtype=torch.float32).to(device)

F:\Programs\Anaconda\lib\site-packages\skopt\sampler\sobol.py:250: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+16386=16386. 
  total_n_samples))
F:\Programs\Anaconda\lib\site-packages\skopt\sampler\sobol.py:250: UserWarning: The balance properties of Sobol' points require n to be a power of 2. 0 points have been previously generated, then: n=0+130=130. 
  total_n_samples))


In [5]:
def train(inn_var, bounds, model, loss, weight, optimizer, scheduler, log_loss):

    ind_bounds1 = torch.isclose(inn_var[:, 0], torch.ones_like(inn_var[:, 0]) * bounds[0][0])
    ind_bounds2 = torch.isclose(inn_var[:, 0], torch.ones_like(inn_var[:, 0]) * bounds[0][1])
    ind_bounds3 = torch.isclose(inn_var[:, 1], torch.ones_like(inn_var[:, 1]) * bounds[1][0])
    ind_bounds4 = torch.isclose(inn_var[:, 1], torch.ones_like(inn_var[:, 1]) * bounds[1][1])

    ind = torch.arange(inn_var.shape[0], dtype=torch.long).to(device)
    ind_inner =ind[~ind_bounds1*~ind_bounds2*~ind_bounds3*~ind_bounds4]

    # if not adaptive or len(log_loss) == 0:
    #     weight = 1.0
    # else:
    #     weight = log_loss[-1][-2]

    def closure():

        optimizer.zero_grad()
        out_var = model(inn_var)
        out_var = model.output_transform(inn_var, out_var)
        res_i, duda, dvda = model.equation(inn_var, out_var)
        eqs_loss_1 = loss(res_i[:, 0], torch.zeros_like(res_i[:, 0], dtype=torch.float32))
        eqs_loss_2 = loss(res_i[:, 1], torch.zeros_like(res_i[:, 1], dtype=torch.float32))
        eqs_loss_3 = loss(res_i[:, 2], torch.zeros_like(res_i[:, 2], dtype=torch.float32))
        # del res_i
        ieq_loss = model.inequal_constrain(out_var[ind_inner])
        opt = model.optim_func(inn_var[ind_inner], out_var[ind_inner], duda[ind_inner], dvda[ind_inner], if_reduced=True)
        opt_loss = opt
        loss_batch = weight[0] * eqs_loss_1 + weight[1] * eqs_loss_2 + weight[2] * eqs_loss_3\
                     + weight[3] * ieq_loss + weight[4] * opt_loss
        loss_batch.backward()

        log_loss.append([eqs_loss_1.item(), eqs_loss_2.item(), eqs_loss_3.item(),
                         ieq_loss.item(), opt_loss.item(), loss_batch.item()])

        return loss_batch

    optimizer.step(closure)
    scheduler.step()


def inference(inn_var, model):

    with torch.no_grad():
        out_pred = model(inn_var)
        out_pred = model.output_transform(inn_var, out_pred)
        out_alph = model.alpha(out_pred[..., -1])

    return out_pred, out_alph

In [6]:
# 建立网络
Net_model = Net(planes=[2] + [64]*4 + [4],).to(device)
# 损失函数
L2loss = nn.MSELoss()
# 优化算法
Optimizer = torch.optim.Adam(Net_model.parameters(), lr=0.001, betas=(0.7, 0.9))
# 下降策略
Scheduler = torch.optim.lr_scheduler.MultiStepLR(Optimizer, milestones=[30000], gamma=0.1)
# 可视化
Visual = matplotlib_vision('/')

Boundary_epoch = [30001, ]
display_epoch = 1000

x_vis = np.linspace(0, 1, 81).astype(np.float32)[:, None]
y_vis = np.linspace(0, 1, 81).astype(np.float32)[:, None]

x_vis = np.tile(x_vis, (1, y_vis.shape[0]))  # Nx x Ny
y_vis = np.tile(y_vis, (1, x_vis.shape[0])).T  # Nx x Ny
input_visual = np.stack((x_vis, y_vis), axis=-1)
input_visual = torch.tensor(input_visual, dtype=torch.float32, requires_grad=True).to(device)


In [ ]:

#
star_time = time.time()
log_loss = []
pred_par = []
mu_f, mu_h = 0.1/3, 1e4
Weight = [mu_f, mu_f, mu_f, mu_h, 1.]

inn_var = input_train
inn_var.requires_grad_(True)
# Training
for iter in range(30001):

    lr_net = Optimizer.state_dict()['param_groups'][0]['lr']

    train(inn_var, [bound_x, bound_y], Net_model, L2loss, Weight, Optimizer, Scheduler, log_loss)

    if iter > 0 and iter % display_epoch == 0:
        print(' iter: {:6d}, lr_net: {:.3e},  cost: {:.2f} , total loss: {:.3e}, '
              'eqs_loss_1: {:.3e}, eqs_loss_2: {:.3e}, eqs_loss_3: {:.3e}, ieq_loss: {:.3e}, opt_loss: {:.3e} '.
              format(iter, lr_net,  time.time() - star_time, log_loss[-1][-1],
                     log_loss[-1][0], log_loss[-1][1], log_loss[-1][2], log_loss[-1][3], log_loss[-1][4],))


        plt.figure(1, figsize=(15, 10))
        plt.clf()
        Visual.plot_loss(np.arange(len(log_loss)), np.array(log_loss)[:, 0], 'eqs_loss_1')
        Visual.plot_loss(np.arange(len(log_loss)), np.array(log_loss)[:, 1], 'eqs_loss_2')
        Visual.plot_loss(np.arange(len(log_loss)), np.array(log_loss)[:, 2], 'eqs_loss_3')
        Visual.plot_loss(np.arange(len(log_loss)), np.array(log_loss)[:, 3], 'ieq_loss')
        Visual.plot_loss(np.arange(len(log_loss)), np.array(log_loss)[:, 4], 'opt_loss')
        Visual.plot_loss(np.arange(len(log_loss)), np.array(log_loss)[:, -1], 'total_loss')
        plt.savefig(res_path + 'log_loss.svg')

        star_time = time.time()

    if iter > 0 and iter % display_epoch == 0:

        output_visual, alpha_visual = inference(input_visual, Net_model)
        output_visual = output_visual.detach().cpu()

        v_visual = torch.sqrt(output_visual[..., 0]**2 + output_visual[..., 1]**2)
        field_visual = torch.stack((v_visual,output_visual[..., -2], output_visual[..., -1], alpha_visual.detach().cpu()), dim=-1).numpy()
        coord_visual = input_visual.detach().cpu().numpy()
        fmin, fmax = field_visual.min(axis=(0, 1)), field_visual.max(axis=(0, 1))
        cmin, cmax = coord_visual.min(axis=(0, 1)), coord_visual.max(axis=(0, 1))
        x_pos = coord_visual[:, :, 0]
        y_pos = coord_visual[:, :, 1]
        Num_fields = field_visual.shape[-1]


        field_name = ['velocity', 'Pressure', 'rho', 'alpha']
        input_name = ['x', 'y']
        font = {'family': 'Times New Roman', 'weight': 'normal', 'size': 20}
        for fi in range(Num_fields):
            plt.figure(fi+1, figsize=(15, 10))
            plt.clf()
            plt.rcParams['font.size'] = 20
            ########      Exact f(t,x,y)     ###########
            # plt.subplot(1, Num_fields,  0 * Num_fields + fi + 1)
            f_true = field_visual[:, :, fi]
            plt.pcolormesh(x_pos, y_pos, f_true, cmap='RdBu_r', shading='gouraud', antialiased=True, snap=True)
            cb = plt.colorbar()
            plt.contour(x_pos, y_pos, f_true, levels=20, linestyles='-', linewidths=0.4, colors='k')
            plt.axis((cmin[0], cmax[0], cmin[1], cmax[1]))
            plt.clim(vmin=fmin[fi], vmax=fmax[fi])
            cb.set_label('Analytical Solution $' + field_name[fi] + '$', rotation=0, fontdict=font, y=1.12)
            # 设置图例字体和大小
            cb.ax.tick_params(labelsize=20)
            for l in cb.ax.yaxis.get_ticklabels():
                l.set_family('Times New Roman')
            tick_locator = ticker.MaxNLocator(nbins=5)  # colorbar上的刻度值个数
            cb.locator = tick_locator
            cb.update_ticks()
            plt.xlabel('$' + input_name[0] + '$', fontdict=font)
            plt.ylabel('$' + input_name[1] + '$', fontdict=font)
            plt.yticks(fontproperties='Times New Roman', size=20)
            plt.xticks(fontproperties='Times New Roman', size=20)
            plt.savefig(res_path + 'field_' + str(field_name[fi]) + '-' + 'now.jpg')
            # plt.savefig(res_path + 'field_' + str(field_name[fi]) + '-' + str(iter) + '.jpg')



        torch.save({'epoch': iter, 'model': Net_model.state_dict(),
                    'log_loss': np.array(log_loss)}, res_path + 'latest_model.pth')



 iter:   1000, lr_net: 1.000e-03,  cost: 363.46 , total loss: 9.998e+01, eqs_loss_1: 7.474e-03, eqs_loss_2: 8.064e-06, eqs_loss_3: 9.399e-03, ieq_loss: 9.993e-03, opt_loss: 4.907e-02 
 iter:   2000, lr_net: 1.000e-03,  cost: 366.14 , total loss: 9.998e+01, eqs_loss_1: 5.830e-03, eqs_loss_2: 2.167e-04, eqs_loss_3: 3.685e-03, ieq_loss: 9.993e-03, opt_loss: 4.891e-02 
 iter:   3000, lr_net: 1.000e-03,  cost: 370.11 , total loss: 9.998e+01, eqs_loss_1: 3.913e-03, eqs_loss_2: 7.346e-04, eqs_loss_3: 4.264e-03, ieq_loss: 9.993e-03, opt_loss: 4.881e-02 
 iter:   4000, lr_net: 1.000e-03,  cost: 364.35 , total loss: 9.998e+01, eqs_loss_1: 2.760e-03, eqs_loss_2: 1.013e-03, eqs_loss_3: 1.259e-03, ieq_loss: 9.993e-03, opt_loss: 4.943e-02 
 iter:   5000, lr_net: 1.000e-03,  cost: 363.81 , total loss: 9.998e+01, eqs_loss_1: 2.345e-03, eqs_loss_2: 8.951e-04, eqs_loss_3: 6.355e-04, ieq_loss: 9.993e-03, opt_loss: 4.921e-02 
 iter:   6000, lr_net: 1.000e-03,  cost: 363.76 , total loss: 9.998e+01, eqs_los